<a href="https://colab.research.google.com/github/drcolula/mtypollution/blob/master/pol_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pollution analysis program:

By: David Ramirez Colula(1), Jesus Alberto Gonzalez Morales(1), Héctor Cruz Rojas(1) and Luis Alberto Chávez Tress(1)

a00818988@itesm.mx

Under the advise of Daniel Astudillo(1) and Luis Garza(1)

(1) Tecnológico de Monterrey, Escuela de Ingeniería y Ciencias

This program uses a neural network for interpolating the PM10 pollution measurements in Monterrey, Mexico from historical data of the seven official measurement stations in the city, with parameters: Air pressure, Relative Humidity, Temperature, Wind Speed and distance between stations

Instructions:

Run the cells under the 'Required' section

For training a new model, run the cells under the 'Model Training' Section.
For using the model that was already trained, only run the cells under the 'Required' and 'Pretrained Model' Sections

*For map visualization, it is required a Mapbox access token, which can be obtained at https://account.mapbox.com/

This code was created and optimized in the Google Colab Framework and can be found at the github repository: https://github.com/drcolula/mtypollution 

#Mapbox Access Token

In [0]:
#For running maps, it is required an access token from mapbox at https://account.mapbox.com/ Paste the token in the next line between the ''
mapboxtoken=''

#Required

In [0]:
#Import libraries
import numpy as np
import geopy.distance
import pandas as pd
import random
import math
import plotly.express as px
import tensorflow as tf

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense

import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split

Using TensorFlow backend.


In [0]:
#Create Folders for outputs
!mkdir tables
!mkdir graphs && cd graphs && mkdir ModeloGeneraltestF
!mkdir maps

In [0]:
#Clone github repository
!git clone https://github.com/drcolula/mtypollution.git

Cloning into 'mtypollution'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [0]:
#Load datasets and Transform to Pandas Data Frames
_data = np.load('mtypollution/_data.npy')
_date = np.load('mtypollution/_date.npy',allow_pickle=True)
dates=pd.DatetimeIndex(_date)
#Create dict with each station
ds={}
for x in range(1,9):
        ds["s{0}".format(x)]=pd.DataFrame(_data[:,x-1,:],columns=['PM10', 'PRS', 'RAIN', 'RH', 'SR', 'T', 'WS'],index=dates)
        ds["s{0}".format(x)] = ds["s{0}".format(x)].loc[~ds["s{0}".format(x)].index.duplicated(keep='first')] #Drop duplicates
pm10=pd.DataFrame()
#Create target values (PM10) table and standarize the rest of the values
for key in ds:
    pm10[key]=ds[key]['PM10']
    ds[key]=((ds[key].drop('PM10',axis=1)-ds['s1'].drop('PM10',axis=1).mean())/ds['s1'].drop('PM10',axis=1).std())
    ds[key].insert(0,'PM10',pm10[key])

In [0]:
#Distance between two stations
def dist(e1,e2):
  station_position=[0]*8
  station_position[0] = (25.668366, -100.249512)
  station_position[1] = (25.742053, -100.266144)
  station_position[2] = (25.676027, -100.335939)
  station_position[3] = (25.757178, -100.365981)
  station_position[4] = (25.675702, -100.464958)
  station_position[5] = (25.783503, -100.585897)
  station_position[6] = (25.802037, -100.343083)
  station_position[7] = (25.777312, -100.187950)
  distx=geopy.distance.distance((station_position[e1-1][0],station_position[e1-1][1]),(station_position[e1-1][0],station_position[e2-1][1])).km
  disty=geopy.distance.distance((station_position[e1-1][0],station_position[e1-1][1]),(station_position[e2-1][0],station_position[e1-1][1])).km
  return(distx,disty)

In [0]:
#Create station parameters dataset
dc=ds['s1'].drop(['RAIN','SR'], axis=1)
for i in range(2,9):
  s=str(i)
  dc=dc.join(ds['s'+s].drop(['RAIN','SR'], axis=1),rsuffix=('_s'+s))
dc.rename(columns={'PM10':'PM10_s1','PRS':'PRS_s1','T':'T_s1','WS':'WS_s1','RH':'RH_s1'},inplace=True)

In [0]:
#Create dataset with three copies of each row
db=dc.append(dc).append(dc).sort_index()

#Model Training

In [0]:
#Random selection of three stations
def targets():
  set1 = {'s1', 's2', 's3', 's4', 's5','s6','s7','s8'} 
  x=random.sample(set1, 3)
  return(x)

In [0]:
#Create random target station for each row
l=[]
for i in range(0,18682):
    t=targets()
    z=0
    for x in t:
        l.append(t[z])
        z=z+1
db['Target']=l

In [0]:
#Compute distance between target station and stations
for i in range(1,9):
    dc['ds'+str(i)+'_x']=dc.Target.apply(lambda x: dist(i,int(x[1]))[0])
    dc['ds'+str(i)+'_y']=dc.Target.apply(lambda x: dist(i,int(x[1]))[1])

In [0]:
#Append PM10 values of target station
db['PM10obj']=None
for i in range(0,56046):
    db['PM10obj'].iloc[i]=ds[db.Target.iloc[i]].loc[db.index[i]].PM10

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
#Export and Drop name of target station
db['Target'].to_csv('tables/targets_training.csv')
db.drop('Target',axis=1,inplace=True)

In [0]:
#Create Target and variables datasets
dobjF=db.PM10obj
dsgF=db.drop('PM10obj',axis=1)

In [0]:
#Export training dataset
dsgF.to_csv('tables/dsgF.csv')
dobjF.to_csv('tables/dobjF.csv')

In [0]:
#Neural network architecture
def modevalnn5(dfx,dfy,ttype):
    x_train, x_test, y_train, y_test = train_test_split(dfx,dfy, test_size=0.20, random_state=42)
    model = Sequential()

    #Input Layer :
    model.add(Dense(240, kernel_initializer='normal',input_dim = dfx.shape[1], activation='relu'))

    #Hidden Layers :
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
   
    #Output Layer :
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))

    #Compilation:
    model.compile(loss='mean_absolute_error', optimizer='rmsprop',  metrics=['mae','mse','acc'])
   

    #Model Fitting
    history = model.fit(x_train,y_train, validation_split=0.2,epochs= 1500, shuffle  = True, batch_size = 10, verbose=1)
   
    history_dict = history.history
    print(history_dict.keys())

    # Mae graph
    mae_values = history.history['mae']
    val_mae_values = history.history['val_mae']
    plt.figure(figsize=(12,5))
    plt.plot(mae_values,'b', label='mae')
    plt.plot(val_mae_values,'g', label='Validation mae')
    plt.title('MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.savefig('./graphs/ModeloGeneraltestF/mae'+'_'+ttype)
    plt.show()
    
    # RMSE graph
    rmse_values = np.sqrt(np.array(history.history['mse']))
    val_rmse_values = np.sqrt(np.array(history.history['val_mse']))
    plt.figure(figsize=(12,5))
    plt.plot(rmse_values,'b', label='RMSE')
    plt.plot(val_rmse_values,'g', label='Validation RMSE')
    plt.title('RMSE')
    plt.ylabel('RMSE')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.savefig('./graphs/ModeloGeneraltestF/RMSE'+'_'+ttype)
    plt.show()
    
    #Acc graph
    acc_values = history_dict['acc']
    val_acc_values = history_dict['val_acc']
    plt.figure(figsize=(12,5))
    plt.plot(acc_values, 'b', label='Accuracy')
    plt.plot(val_acc_values, 'g', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper left')
    plt.savefig('./graphs/ModeloGeneraltestF/Acc'+'_'+ttype)
    plt.show()

   # Model Evaluation
    scores = model.evaluate(x_test, y_test, verbose=0)
    scores[2]=math.sqrt(scores[2])
    print(pd.DataFrame([scores], columns=model.metrics_names))

    #Saving model
    model.save('mods_'+ttype+'.hdf5')
    

In [0]:
#Model Training
modevalnn5(dsgF,dobjF,'ModGNormF128')

#Pretrained Model

In [0]:
#X and y distance components between a target station and a point (lat,lon)
def distpt(e,p):
  station_position=[0]*8
  station_position[0] = (25.668366, -100.249512)
  station_position[1] = (25.742053, -100.266144)
  station_position[2] = (25.676027, -100.335939)
  station_position[3] = (25.757178, -100.365981)
  station_position[4] = (25.675702, -100.464958)
  station_position[5] = (25.783503, -100.585897)
  station_position[6] = (25.802037, -100.343083)
  station_position[7] = (25.777312, -100.187950)
  distx=geopy.distance.distance((station_position[e-1][0],station_position[e-1][1]),(station_position[e-1][0],p[1])).km
  disty=geopy.distance.distance((station_position[e-1][0],station_position[e-1][1]),(p[0],station_position[e-1][1])).km
  return(distx,disty)

In [0]:
#Create point grid, allocating w points between x coordinates and h points
#between y coordinates; and attach distance between each point and each 
#target station
def meshing(w,h):
    spacingx = np.linspace(-100.635239, -100.092657, w)
    spacingy = np.linspace(25.618419, 25.830345, h)
    X2 = np.meshgrid(spacingy, spacingx)
    grid_shape = X2[0].shape
    X2 = np.reshape(X2, (2, -1)).T
    co=pd.DataFrame(X2,columns=['lat','lon'])
    cc=pd.DataFrame()
    cc['coor'] = list(zip(co.lat, co.lon))
    for i in range(1,9):
        cc['ds'+str(i)+'_x']=cc.coor.apply(lambda x: distpt(i,x)[0])
        cc['ds'+str(i)+'_y']=cc.coor.apply(lambda x: distpt(i,x)[1])
    return(cc,co)

In [0]:
#Create grid
w=300
h=150
(cc,co)=meshing(w,h)

CPU times: user 3min 6s, sys: 33.6 ms, total: 3min 6s
Wall time: 3min 6s


In [0]:
#Predict points in the mesh for the n first hours and save the result
n=10000

mod=load_model('mtypollution/mods_ModGNormF128.hdf5')
    
for i in range(0,n):
    dd=pd.concat([dc.iloc[i,:]]*(w*h),axis=1).T
    ff=pd.concat([dd.reset_index(),cc.drop('coor',axis=1).reset_index()],axis=1).drop('index',axis=1)
    pred=mod.predict(ff)
    co['PM10']=pred
    co['hr']=list(dc.index)[i]
    co.to_csv('maps/' + str(dc.index[i]).replace(':','-') + '.csv',index=False)

CPU times: user 27.5 s, sys: 1.77 s, total: 29.3 s
Wall time: 36 s


In [0]:
#Visualization example with plotly and mapbox
maptest=pd.read_csv('maps/2012-01-01 00-00-00.csv')
mapboxtoken=''
px.set_mapbox_access_token(mapboxtoken)
fig=px.scatter_mapbox(test,lat='lat',lon='lon',color=pred)
fig.update_layout(mapbox_accesstoken=mapboxtoken)
fig.show()